# process "/obs1/gps/rtkfix"  "/velodyne_points" and "/image_raw" 
- 生成 obs1_rtkfix as csv
- 生成 top_bird_view as numpy.array
- 生成 image as numpy.array and jpg

In [1]:
from __future__ import print_function
import sensor_msgs.point_cloud2 as pc2
import numpy as np
import rosbag
import os
from PIL import Image
import cv2
import math
from collections import defaultdict
import pandas as pd
# ROSBAG SETTINGS
data_dir = "/home/dl/db"

from tqdm import tqdm
def process_msg(bag_name):
    bag_file = os.path.join(data_dir, bag_name)

    # OPEN ROSBAG
    bag = rosbag.Bag(bag_file, "r")
    pcl_messages = bag.read_messages(topics=["/velodyne_points"])
    n_lidar = bag.get_message_count(topic_filters=["/velodyne_points"])
    
    img_messages = bag.read_messages(topics=["/image_raw"])
    n_img = bag.get_message_count(topic_filters=["/image_raw"])
    skip_cycle = 20  # generate images
    
    
    obs_messages = bag.read_messages(topics=['/obs1/gps/rtkfix'])
    n_obs = bag.get_message_count(topic_filters=['/obs1/gps/rtkfix'])
    obs_dicts=defaultdict(list)
    rtk_cols=["timestamp", "rx", "ry", "rz", "lx", "ly", "lz"]
    pbar = tqdm(total=n_obs)
    
    for i in range(n_obs):
        pbar.update(1)
        
        # READ NEXT MESSAGE IN BAG
        topic, msg, t  = obs_messages.next()
        rtk2dict(msg,obs_dicts)
    pbar.close()
    obs_rtk_df = pd.DataFrame(data=obs_dicts, columns=rtk_cols)
    obs_rtk_df.to_csv(os.path.join(data_dir+'/out_dir/'+bag_name.split('.')[0]+'/%s_rtk.csv' % bag_name), index=False)

    
    
    
    
    
    pbar = tqdm(total=n_img)
   
    for i in range(n_img):
        pbar.update(1)
        
        # READ NEXT MESSAGE IN BAG
        topic, msg, t  = img_messages.next()

        # CONVERT MESSAGE TO A NUMPY ARRAY
        img = np.fromstring(msg.data, dtype=np.uint8)
        img = img.reshape(msg.height, msg.width)

        # CONVERT TO RGB
        img = cv2.cvtColor(img, cv2.COLOR_BayerGR2BGR)

        # DO SOME PROCESSING ON THE IMAGE    
        seqimg=msg.header.stamp.to_nsec()
        save_np_csv(bag_name,img,seqimg,'/image_raw',saveimg=i % skip_cycle == 0)
    pbar.close()

    

    
    pbar = tqdm(total=n_lidar)
    for i in range(n_lidar):
        pbar.update(1)
        # READ NEXT MESSAGE IN BAG
        topic, msg, t = pcl_messages.next()

        # CONVERT MESSAGE TO A NUMPY ARRAY OF POINT CLOUDS
        # creates a Nx5 array: [x, y, z, reflectance, ring]
        lidar = pc2.read_points(msg)
        lidar = np.array(list(lidar))

        # PROCESS THE POINT CLOUDS
        top=pcl2top(lidar)
        seqtop=msg.header.stamp.to_nsec()
        save_np_csv(bag_name,top,seqtop,'/lidar')
    pbar.close()
    

    



    

#  process pcl to top_bird_view 
- 数组形状: height,width,channel=400,400,8

- 选择矩形区域height,width=40,40
- 遍历所有点
- 将稀疏空间划分为网格空间 0.1x0.1x6切片
- 生成六层的height map
- 生成intensity map 
- 生成mass map 


In [2]:

TOP_Y_MIN=-20  #40  ###<todo> determine the correct values!
TOP_Y_MAX=+20      ###<todo> determine the correct values!
TOP_X_MIN=0
TOP_X_MAX=40   #70.4 ###<todo> determine the correct values!
TOP_Z_MIN=-2.0    ###<todo> determine the correct values!
TOP_Z_MAX= 0.4    ###<todo> determine the correct values!

TOP_X_DIVISION=0.1  #0.1
TOP_Y_DIVISION=0.1
TOP_Z_DIVISION=0.4
def pcl2top(points):
    X0, Xn = 0, int((TOP_X_MAX-TOP_X_MIN)//TOP_X_DIVISION)+1
    Y0, Yn = 0, int((TOP_Y_MAX-TOP_Y_MIN)//TOP_Y_DIVISION)+1
    Z0, Zn = 0, int((TOP_Z_MAX-TOP_Z_MIN)//TOP_Z_DIVISION)+1
    height  = Yn - Y0
    width   = Xn - X0
    channel = Zn - Z0  + 2
    pxs=points[:,0]
    pys=points[:,1]
    pzs=points[:,2]
    prs=points[:,3]
    tx=lambda x: ((x-TOP_X_MIN)//TOP_X_DIVISION).astype(np.int32)
    ty=lambda y: ((y-TOP_Y_MIN)//TOP_Y_DIVISION).astype(np.int32)
    tz=lambda z: ((z-TOP_Z_MIN)//TOP_Z_DIVISION).astype(np.int32)
    nz=lambda z: ((z-TOP_Z_MIN)/TOP_Z_DIVISION)
#     print('height,width,channel=%d,%d,%d'%(height,width,channel))
    top = np.zeros(shape=(height,width,channel), dtype=np.float32)
    ## start to make top  here !!!
    ret=np.where((points[:,0]> TOP_X_MIN) & \
                 (points[:,0]<TOP_X_MAX) & \
                 (points[:,1]> TOP_Y_MIN) & \
                 (points[:,1]<TOP_Y_MAX) & \
                 (points[:,2]> TOP_Z_MIN) & \
                 (points[:,2]<TOP_Z_MAX) 
                )
    ret_points=points[ret]
    for i in ret_points:
        top[tx(i[0])][ty(i[1])][Zn+1]+=1
        if nz(i[2]) >top[tx(i[0])][ty(i[1])][tz(i[2])]:
            top[tx(i[0])][ty(i[1])][tz(i[2])]=nz(i[2])
            top[tx(i[0])][ty(i[1])][Zn]=i[3]
    top[:,:,Zn+1] = np.log(top[:,:,Zn+1]+1)/math.log(64)
        
    return top


In [3]:
def save_np_csv(bag_name,data,seq,topic,saveimg=False):
#     print bag_name,data.shape,seq
    out_dir=data_dir+'/out_dir/'+bag_name.split('.')[0]+topic
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    np.save(out_dir+'/%05d.npy'%seq,data)
    if saveimg:
        cv2.imwrite(out_dir+'/%05d.jpg'%seq, data)
        

    

In [4]:
def rtk2dict(msg,rtk_dict):
    
    rtk_dict["timestamp"].append(msg.header.stamp.to_nsec())

    rtk_dict["lx"].append(msg.twist.twist.linear.x) 
    rtk_dict["ly"].append(msg.twist.twist.linear.y) 
    rtk_dict["lz"].append(msg.twist.twist.linear.z)
    rtk_dict["rx"].append(msg.twist.twist.angular.x) 
    rtk_dict["ry"].append(msg.twist.twist.angular.y) 
    rtk_dict["rz"].append(msg.twist.twist.angular.z)
    

# 遍历所有bag

In [5]:
def get_bags(data_dir):
    bags=[]
    for root, dirs, files in os.walk(data_dir):
        for basename in files:
            if basename.split('.')[1]=='bag':
                bags.append(basename)
    return bags
bags=get_bags(data_dir)
for bag_name in bags:
    process_msg(bag_name)


100%|██████████| 120/120 [03:09<00:00,  1.77s/it]
